In [0]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [0]:
import numpy as np
import pandas as pd 
import os
# import git
import librosa
import matplotlib.pyplot as plt
import gc

from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import zipfile

tqdm.pandas()

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

In [0]:
!unzip "/content/drive/My Drive/Augmented_Features.zip"

Archive:  /content/drive/My Drive/Augmented_Features.zip
   creating: Augmented_Features/
  inflating: Augmented_Features/albanian1.pkl  
  inflating: Augmented_Features/albanian10.pkl  
  inflating: Augmented_Features/albanian11.pkl  
  inflating: Augmented_Features/albanian12.pkl  
  inflating: Augmented_Features/albanian13.pkl  
  inflating: Augmented_Features/albanian14.pkl  
  inflating: Augmented_Features/albanian15.pkl  
  inflating: Augmented_Features/albanian16.pkl  
  inflating: Augmented_Features/albanian17.pkl  
  inflating: Augmented_Features/albanian18.pkl  
  inflating: Augmented_Features/albanian19.pkl  
  inflating: Augmented_Features/albanian2.pkl  
  inflating: Augmented_Features/albanian20.pkl  
  inflating: Augmented_Features/albanian21.pkl  
  inflating: Augmented_Features/albanian22.pkl  
  inflating: Augmented_Features/albanian23.pkl  
  inflating: Augmented_Features/albanian24.pkl  
  inflating: Augmented_Features/albanian25.pkl  
  inflating: Augmented_Feature

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv("drive/My Drive/aug_speakers.csv")

In [0]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')
df = pd.read_csv("drive/My Drive/aug_speakers.csv")
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()


Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?'],
      dtype='object')


,age,age_onset,speakerid
count,3621.000000,3621.000000,3621.000000
mean,32.852251,10.044877,1073.967688
std,14.022968,8.579382,628.435246
min,0.000000,0.000000,1.000000
25%,22.000000,5.000000,538.000000
50%,28.000000,10.000000,1048.000000
75%,41.000000,13.000000,1616.000000
max,97.000000,86.000000,2176.000000


In [0]:
import collections
samples_count = collections.Counter(df["native_language"].values)
new_d = {}
for key,val in samples_count.items():
    new_d[val] = new_d.get(val,[])+[key]

for i in range(8):
    if i in new_d.keys():
        del new_d[i]

# print(new_d)
sorted_d = {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}
classes = []
for _,val in sorted_d.items():
    classes+=val
sorted_d.clear()
new_d.clear()

# print(total)
# print(len(classes))
# {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}

In [0]:
# import csv
# def run_function(func,data,sampling_rate,age,sex):
#     if func == "pitch":
#         if sex == "male":
#             y = librosa.effects.pitch_shift(data, sampling_rate, n_steps=4)
#         else:
#             y = librosa.effects.pitch_shift(data, sampling_rate, n_steps=-4)
#     elif func == "faster":
#         y = librosa.effects.time_stretch(data, 2.0)
#     elif func == "slower":
#         y = librosa.effects.time_stretch(data, 0.5)
#     return y


# def trim_pad(data):
#     sr_dur = 22050*18
#     if 0 < len(data): 
#         y, _ = librosa.effects.trim(data) 
#         trim_long_data = True
#         if len(y) > sr_dur: 
#             if trim_long_data:
#                 y = y[0:0+sr_dur]
#         else: 
#             padding = sr_dur - len(y)    
#             offset = padding // 2
#             y = np.pad(y, (offset, sr_dur - len(y) - offset), 'constant')
#         return y
#     return data
     

# def append_new_row_to_df(new_row_data,file_counter):   
# #     print("in append_new_data_to_df")
# #     print(new_row_data)
#     with open('drive/My Drive/aug_speakers.csv','a',newline='') as f:
#         writer=csv.writer(f)
#         new_row_data[3] = new_row_data[4]+str(file_counter)
#         writer.writerow(new_row_data)
# #     print("out append_new_data_to_df")
    

# func_list = ["pitch","faster","slower"]
# except_flag = False
# for j,label in enumerate(classes):
#     print(j+1,label)
#     lang_df = df.loc[df['native_language'] == label]
#     row_datas = []
#     for _,row in lang_df.iterrows():
#         if not row['file_missing?']:
#             try:
#                 path = 'recordings/' + row['filename'] +".mp3"
#                 data, sampling_rate = librosa.load(path, sr=SAMPLE_RATE, duration=18)
#                 row_datas.append([data,sampling_rate,row.values])
#             except Exception:
#                 print("exception at filename "+row['filename']) 

#     row_datas_exists = len(row_datas)
#     if row_datas_exists: #Means there was no exception when loading the file
#         for func in func_list:
#             i = 0
#             new_row_datas = []
#             while i < len(row_datas) and (len(row_datas)+i) < 50:
#                 new_data = run_function(func,row_datas[i][0],row_datas[i][1],row_datas[i][2][0],row_datas[i][2][5])
#                 new_row_datas.append([new_data,row_datas[i][1],row_datas[i][2].copy()])
#                 append_new_row_to_df(row_datas[i][2].copy(),len(row_datas)+i+1) #define this function
#                 i+=1
#             row_datas += new_row_datas
#         row_datas = row_datas[:50]
#         print(len(row_datas))
#         pickle_counter = 0
#         for row_data in row_datas:
#             trim_padded_y = trim_pad(row_data[0])
#             mfcc = librosa.feature.mfcc(y=trim_padded_y, sr=sampling_rate, n_mfcc = 40)
#             melspec = librosa.feature.melspectrogram(trim_padded_y, sr=sampling_rate, n_mels=128)
#             save_data={"data": trim_padded_y,"sampling_rate":sampling_rate,"mfcc": mfcc,"melspec":melspec}
#             pickle_counter+=1
#             with open('Augmented_Features/'+row['native_language']+str(pickle_counter)+'.pkl', 'wb') as outfile:
#                 pickle.dump(save_data, outfile, pickle.HIGHEST_PROTOCOL)

In [0]:
start_time = time.time()

def extract_features_from_pkl(fname):
    try:
        path = 'Augmented_Features/' + fname +".pkl"
        extracted_data = pd.read_pickle(path)
        data, sampling_rate = extracted_data["data"], extracted_data["sampling_rate"]
        mfcc = extracted_data["mfcc"]
        melspec = extracted_data["melspec"]
       
        return (mfcc,melspec,data,sampling_rate)
    except Exception:
        return 
mfccs,melspecs,datas,sampling_rates = [],[],[],[]
for _, row in df.iterrows():
    print(_)
    filename = row['filename']
    val = extract_features_from_pkl(filename)
    if val:
        mfccs.append(val[0])
        melspecs.append(val[1])
        datas.append(val[2])
        sampling_rates.append(val[3])
    else:
        mfccs.append(None)
        melspecs.append(None)
        datas.append(None)
        sampling_rates.append(None)
        

print("time taken "+str((time.time() - start_time)/60))

In [0]:
df.head()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?
0,24.0,12.0,"koussi, senegal",balanta,balanta,male,788,senegal,True
1,18.0,10.0,"buea, cameroon",cameroon,cameroon,male,1953,cameroon,True
2,48.0,8.0,"hong, adamawa, nigeria",fulfulde,fulfulde,male,1037,nigeria,True
3,42.0,42.0,"port-au-prince, haiti",haitian,haitian,male,1165,haiti,True
4,40.0,35.0,"port-au-prince, haiti",haitian,haitian,male,1166,haiti,True


In [0]:
df["mfcc"] = mfccs
df["melspec"] = melspecs

In [0]:
mfccs = None
melspecs = None

In [0]:
df = df.dropna()
df.shape

(2550, 11)

In [0]:
df.drop(columns=['age_onset','filename','speakerid','file_missing?'],inplace=True)
df.tail()

,age,birthplace,native_language,sex,country,mfcc,melspec
3616,41.0,"brugge, belgium",dutch,male,belgium,"[[-563.0171754320014, -563.0171754320014, -563...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.42..."
3617,37.0,"bahia, brazil",portuguese,female,brazil,"[[-510.4300054248348, -507.23635409843996, -48...","[[0.027289903971678055, 0.012839436725380694, ..."
3618,21.0,"novo hamburgo, rio grande do sul, brazil",portuguese,male,brazil,"[[-602.1734687961401, -602.8813019916938, -603...","[[0.09775744475957422, 0.09866733253236762, 0...."
3619,37.0,"nizhni novgorod, russia",russian,male,russia,"[[-152.2589788337052, -155.56404520604448, -16...","[[0.45721051482387787, 0.36259190909162325, 2...."
3620,20.0,"moscow, russia",russian,female,russia,"[[-352.85362569517923, -352.8802798323855, -35...","[[25.123949210001808, 27.98976061625398, 27.30..."


In [0]:
labels = np.sort(np.unique(df['native_language'].values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c


def test(index):
    global num_class
    labels_array = np.zeros((num_class))
    labels_array[index] = 1
    return labels_array

one_hot_labels = [test(c2i[x]) for x in df['native_language'].values]
df["native_lang_c2i"] = one_hot_labels
df["native_lang_c2i"].head()
for f in labels:
    df[f] = 0.0
    #train_noisy[f] = 0.0

df[labels] = one_hot_labels
df.tail()

one_hot_labels = None
labels = None


In [0]:
df["mfcc"].iloc[32].shape

(40, 776)

In [0]:
class conf:
    # Preprocessing settings
    sampling_rate = 44100
    duration = 4
    hop_length = 300
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration

In [0]:
xmin,ymin = 128,float("inf")
for _,row in df.iterrows():
    x,y = row["melspec"].shape
    xmin,ymin = min(x,xmin),min(y,ymin)
print(xmin,ymin)

128 776


In [0]:
df["mfcc"].iloc[32][:,:680].shape

(40, 680)

In [0]:
def get_melspec(df):
    X = []
    for i, row in df.iterrows():
        x = row["melspec"]
        x = x[:,:680]
        # x = librosa.power_to_db(x)
        # x = x.astype(np.float32)
        X.append(x.transpose())
    return X

X = np.array(get_melspec(df))
X.shape

(2550, 680, 128)

In [0]:
Y = np.array(df.iloc[:,-51:])

In [0]:
df.drop(df.index, inplace=True)

In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [0]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [0]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('linear')
  
  else:
    projection=tf.identity(inputs)
    print('none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [0]:
X_train,X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=27)
X_train,X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.25,random_state=27)

In [0]:
projection=['linear']
accuracy=[]
proj_implemented=[]

In [0]:
for i in projection:
  NUM_LAYERS = 2
  D_MODEL = X.shape[2]
  NUM_HEADS = 4
  UNITS = 64 #1024
  DROPOUT = 0.3
  TIME_STEPS= X.shape[1]
  OUTPUT_SIZE=51
  EPOCHS = 50
  EXPERIMENTS=5

  
  for j in range(EXPERIMENTS):
    model = transformer(time_steps=TIME_STEPS,
      num_layers=NUM_LAYERS,
      units=UNITS,
      d_model=D_MODEL,
      num_heads=NUM_HEADS,
      dropout=DROPOUT,
      output_size=OUTPUT_SIZE,  
      projection=i  )
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    history=model.fit(X_train,Y_train, #batch_size = 32, 
                      epochs=EPOCHS, validation_data=(X_test, Y_test))
    
    y_preds = model.predict(X_test)
    score = model.evaluate(X_test, Y_test,  verbose=1)
    print("score",score)
    accuracy.append(max(history.history['val_accuracy']))
    proj_implemented.append(i)
    del model
    del history

In [0]:
accuracy=pd.DataFrame(accuracy, columns=['accuracy'])
proj_implemented=pd.DataFrame(proj_implemented, columns=['projection'])
results=pd.concat([accuracy,proj_implemented],axis=1)

In [0]:
results.groupby('projection').mean()

,accuracy
projection,
linear,0.105098


In [0]:
results.groupby('projection').std()

,accuracy
projection,
linear,0.005649
